In [3]:
# 示例代码
# coding=utf-8
from multiprocessing import Pipe, Process


def son_process(x, pipe):
    _out_pipe, _in_pipe = pipe

    # 关闭fork过来的输入端
    _in_pipe.close()
    while True:
        try:
            msg = _out_pipe.recv()
            print(msg)
        except EOFError:
            # 当out_pipe接受不到输出的时候且输入被关闭的时候，会抛出EORFError，可以捕获并且退出子进程
            break


if __name__ == '__main__':
    out_pipe, in_pipe = Pipe(True)
    son_p = Process(target=son_process, args=(100, (out_pipe, in_pipe)))
    son_p.start()

    # 等pipe被fork 后，关闭主进程的输出端
    # 这样，创建的Pipe一端连接着主进程的输入，一端连接着子进程的输出口
    out_pipe.close()
    for x in range(1000):
        in_pipe.send(x)
    in_pipe.close()
    son_p.join()
    print ("主进程也结束了")

主进程也结束了


In [1]:
import pandas as pd

In [ ]:
'''
把dictd = {'A':0}转换成DataFrame， 
首先，DataFrame的语法格式应为：

import pandas as pd
df = pd.DataFrame({'A':[0]})#'A'是columns，对应的是list
1
2
输出:

   A
0  0
1
2
但是如果是：

df = pd.DataFrame({'A':0})#直接输入dict
1
会报错

ValueError: If using all scalar values, you must pass an index
1
解决办法1： 
指定index

pd.DataFrame({'A':0}，index=[0])
1
解决办法2： 
把字典放在list中

pd.DataFrame([{'A':0},{'A':1}])
1
输出：

   A
0  0
1  1
'''

In [3]:

pd.DataFrame([{'A':0,'B':1}])

,A,B
0,0,1


In [4]:
pd.DataFrame({'A':0,'B':1},index=[0])

,A,B
0,0,1


In [5]:
import numpy as np
np.array(np.array([0,1,2,3,4,5]))

array([0, 1, 2, 3, 4, 5])

In [1]:
import tensorflow as tf
import numpy as np

In [3]:
normal_dist = tf.distributions.Normal([0.2,0.5,0.3], [0.2,0.5,0.3])

In [6]:
action = tf.clip_by_value(
            normal_dist.sample(1), [0,0,0], [1,1,1])

In [7]:
action

<tf.Tensor 'clip_by_value:0' shape=(1, 3) dtype=float32>

In [10]:
normal_dist.log_prob([[1,2,3],[1,2,3],[1,2,3],[1,2,3]])

<tf.Tensor 'Normal_1/log_prob_2/sub:0' shape=(4, 3) dtype=float32>

In [ ]:
self.mu, self.sigma = tf.squeeze(mu*2), tf.squeeze(sigma+0.1)


In [ ]:
class Actor(object):
    def __init__(self, sess, n_features, n_actions, n_action_bounds, lr=0.0001):

        self.sess = sess
        self.s = tf.placeholder(tf.float32, [1, n_features], "state")
        self.a = tf.placeholder(tf.float32, None, name="act")
        self.td_error = tf.placeholder(
            tf.float32, None, name="td_error")  # TD_error

        l1 = tf.layers.dense(
            inputs=self.s,
            units=50,  # number of hidden units
            activation=tf.nn.relu,
            kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
            bias_initializer=tf.constant_initializer(0.1),  # biases
            name='l1'
        )

        mu = tf.layers.dense(
            inputs=l1,
            units=n_actions,  # output units
            activation=tf.nn.tanh,
            kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
            bias_initializer=tf.constant_initializer(0.1),  # biases
            name='mu'
        )

        sigma = tf.layers.dense(
            inputs=l1,
            units=n_actions,  # output units
            activation=tf.nn.softplus,  # get action probabilities
            kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
            bias_initializer=tf.constant_initializer(1.),  # biases
            name='sigma'
        )
        global_step = tf.Variable(0, trainable=False)
        # self.e = epsilon = tf.train.exponential_decay(2., global_step, 1000, 0.9)
        self.mu, self.sigma = tf.squeeze(mu*2), tf.squeeze(sigma+0.1)
        self.normal_dist = tf.distributions.Normal(self.mu, self.sigma)

        self.action = tf.clip_by_value(
            self.normal_dist.sample(1), action_bound[0], action_bound[1])

        with tf.name_scope('exp_v'):
            log_prob = self.normal_dist.log_prob(
                self.a)  # loss without advantage
            # advantage (TD_error) guided loss
            self.exp_v = log_prob * self.td_error
            # Add cross entropy cost to encourage exploration
            self.exp_v += 0.01*self.normal_dist.entropy()

        with tf.name_scope('train'):
            self.train_op = tf.train.AdamOptimizer(
                lr).minimize(-self.exp_v, global_step)    # min(v) = max(-v)

    def learn(self, s, a, td):
        s = s[np.newaxis, :]
        feed_dict = {self.s: s, self.a: a, self.td_error: td}
        _, exp_v = self.sess.run([self.train_op, self.exp_v], feed_dict)
        return exp_v

    def choose_action(self, s):
        s = s[np.newaxis, :]
        # get probabilities for all actions
        return self.sess.run(self.action, {self.s: s})

In [18]:
import numpy as np

In [19]:
ar = np.array([1.22222,5.222])

In [20]:
"%f,%f" %tuple(ar.tolist())

'1.222220,5.222000'

In [7]:
tuple([1.22222,5.222])

(1.22222, 5.222)

In [1]:
import tensorflow as tf

In [7]:
sess = tf.InteractiveSession()

In [8]:
normal_dist = tf.distributions.Normal([0.1,2.,6.], [1.,1.,1.])

In [ ]:
normal_dist.

In [21]:
log_prop=normal_dist.log_prob([0.05,1.5,5.5])

In [22]:
log_prop.eval()

array([-0.92018855, -1.0439385 , -1.0439385 ], dtype=float32)